<a href="https://colab.research.google.com/github/arkwith7/aSSIST_ML/blob/main/LLM%ED%95%99%EC%8A%B5%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%83%9D%EC%84%B1Genstruct%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genstruct 소개
고품질의 합성 명령 데이터를 생성하는 것은 중요한 과제입니다. 표준 접근 방식은 상황 내 학습과 명령 쌍 생성을 위한 대규모 언어 모델 프롬프트에 크게 의존합니다. 이는 품질, 다양성, 명시적인 추론 부족이라는 측면에서 한계가 있습니다.

이러한 순진한 프롬프트 접근 방식을 개선하기 위한 두 가지 이전 방법은 다음과 같습니다.
- 검색 증강 생성(RAG) 파이프라인은 Wikipedia와 같은 소스의 구절을 지침 쌍으로 변환합니다.
-  대신 [Ada-Instruct](https://arxiv.org/abs/2310.04484)는 프롬프트에 의존하는 대신 지침을 생성하도록 사용자 지정 모델을 교육합니다. 이는 메시지만 표시하는 것에 비해 품질과 다양성을 향상시킵니다. 또한 Ada-Instruct 논문의 저자는 단 10개의 예제로도 훈련을 수행할 수 있다는 사실을 발견했습니다.

Genstruct는 이러한 이전 접근 방식을 결합하고 확장하는 새로운 방법입니다. Ada-instruct와 마찬가지로 프롬프트에 의존하지 않고 맞춤형으로 훈련된 모델입니다. 그러나 Ada-Instruct는 근거 없는 생성에 크게 의존하므로 환각을 일으킬 수 있습니다. 이를 완화하기 위해 Genstruct는 RAG 방법과 같이 사용자가 제공한 컨텍스트를 기반으로 지침을 생성합니다.

또한 Genstruct는 직접적인 질문과 응답보다는 복잡한 질문 생성과 생성된 각 명령 쌍에 대한 다단계 추론에 중점을 두어 이전 작업을 뛰어넘습니다.

## 명령어 쌍 생성(Generating instruction pairs)
Ada-Instruct는 Mistral을 기반으로 훈련됩니다. 특히, 수학이 많이 사용되는 topcs로 추론을 개선하기 위해 [MetaMath-Mistral-7B](meta-math/MetaMath-Mistral-7B) 모델을 통해 훈련되었습니다.

다른 Mistral 모델과 마찬가지로 Huggingface Hub에서 다음과 같이 가져올 수 있습니다.

In [1]:
!pip uninstall transformers==4.35.0
!pip uninstall torch==2.0.1
!pip uninstall sentencepiece==0.1.99
!pip uninstall tokenizers==0.13.3
!pip uninstall accelerate==0.21.0
!pip uninstall bitsandbytes==0.40.0
!pip uninstall vllm
# !pip install fraction
# !pip install protobuf


Found existing installation: transformers 4.43.1
Uninstalling transformers-4.43.1:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.43.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.43.1
Found existing installation: torch 2.0.1+cu117
Uninstalling torch-2.0.1+cu117:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.10/dist-packages/functorch/*
    /usr/local/lib/python3.10/dist-packages/nvfuser/*
    /usr/local/lib/python3.10/dist-packages/torch-2.0.1+cu117.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch/*
    /usr/local/lib/python3.10/dist-packages/torchgen/*
Proceed (Y/n)? Y
  Successfully uninstalled torch-2.0.1+cu117
Found existing installation: sentencepiece 0.1.99
Uninstalling sentencepiece-0.1.99:
  Would remove:
    /

In [1]:
!pip install torch==2.0.1
!pip install transformers==4.35.0
!pip install sentencepiece==0.1.99
!pip install tokenizers==0.13.3
!pip install bitsandbytes==0.40.0
!pip install accelerate==0.21.0


  Using cached transformers-4.35.0-py3-none-any.whl (7.9 MB)
  Using cached tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.8 MB)
  Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.1
    Uninstalling huggingface-hub-0.24.1:
      Successfully uninstalled huggingface-hub-0.24.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.43.1
    Uninstalling transformers-4.43.1:
      Successfully uninstalled transformers-4.43.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.20.0 requires huggingface-hub>=0.21.2, but you 

In [4]:
!pip install transformers -U

  Using cached transformers-4.43.1-py3-none-any.whl (9.4 MB)
  Using cached huggingface_hub-0.24.1-py3-none-any.whl (417 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.0
    Uninstalling transformers-4.35.0:
      Successfully uninstalled transformers-4.35.0


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = 'NousResearch/Genstruct-7B'

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map='cuda', load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

ImportError: tokenizers>=0.14,<0.15 is required for a normal functioning of this module, but found tokenizers==0.13.3.
Try: pip install transformers -U or pip install -e '.[dev]' if you're working with git main

Genstruct는 사용자가 제공한 컨텍스트와 제목에서 지침과 답변을 생성하여 작동합니다. 다음 예와 같이 사용자 정의 프롬프트 형식을 활용합니다.
```
[[[Title]]] p-value
[[[Content]]] The p-value is used in the context of null hypothesis testing in order to quantify the statistical significance of a result, the result being the observed value of the chosen statistic T {\displaystyle T}.[note 2] The lower the p-value is, the lower the probability of getting that result if the null hypothesis were true. A result is said to be statistically significant if it allows us to reject the null hypothesis. All other things being equal, smaller p-values are taken as stronger evidence against the null hypothesis.

The following is an interaction between a user and an AI assistant that is related to the above text.

[[[User]]]
```

그런 다음 모델은 '[[User]]]'에서 완료되어 명령과 응답을 생성합니다.


사용을 단순화하기 위해 Genstruct 토크나이저에는 '채팅 템플릿'이 포함되어 있습니다. 다음에서 생성할 컨텍스트의 제목과 콘텐츠에 대해 'title' 및 'content' 멤버가 포함된 단일 사전을 포함하는 목록을 허용합니다.

In [4]:
msg =[{
    'title': 'p-value',
    'content': "The p-value is used in the context of null hypothesis testing in order to quantify the statistical significance of a result, the result being the observed value of the chosen statistic T {\displaystyle T}.[note 2] The lower the p-value is, the lower the probability of getting that result if the null hypothesis were true. A result is said to be statistically significant if it allows us to reject the null hypothesis. All other things being equal, smaller p-values are taken as stronger evidence against the null hypothesis."
}]
inputs = tokenizer.apply_chat_template(msg, return_tensors='pt').cuda()

그런 다음 `model.generate()`를 사용하여 다음과 같이 생성을 수행할 수 있습니다(또는 vllm 또는 원하는 다른 파이프라인을 사용하여).

In [5]:
gen = tokenizer.decode(model.generate(inputs, max_new_tokens=512)[0]).split(tokenizer.eos_token)[0]
print(gen)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


AttributeError: 'NoneType' object has no attribute 'device'

이 모델은 Wikipedia 기사의 단일 단락 추출에 최적화되어 있습니다. 다른 입력 유형에 따라 운이 달라질 수 있습니다.

## 보상 모델을 사용하여 출력 필터링(Filtering outputs using a reward model)
모델은 때때로 부정확하거나 부적절한 형식의 출력을 생성할 수 있습니다. 보상 모델을 사용한 거부 샘플링이나 간단한 정규식 필터링과 같은 영리한 샘플링 방법을 사용하면 이러한 가능성을 줄일 수 있습니다.

예를 들어 'OpenAssistant/reward-model-deberta-v3-large-v2'를 보상 모델로 간주하고 n개 최고 샘플링을 수행할 수 있습니다.

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

N = 4

rm_tokenizer = AutoTokenizer.from_pretrained('OpenAssistant/reward-model-deberta-v3-large-v2')
rm_model = AutoModelForSequenceClassification.from_pretrained('OpenAssistant/reward-model-deberta-v3-large-v2', torch_dtype=torch.bfloat16)

def extract_pair(resp):
    response = resp.split('[[[Content]]]')[1]
    inst, resp = resp.split('[[[User]]]')[:2]
    return inst.strip(), resp.strip()

def score(resp):
    inst, resp = extract_pair(resp.split(tokenizer.eos_token)[0])

    with torch.no_grad():
        inputs = rm_tokenizer(inst, resp, return_tensors='pt')
        score = float(rm_model(**inputs).logits[0].cpu())
        return score

gens = tokenizer.batch_decode(model.generate(inputs, max_new_tokens=256, num_return_sequences=N, do_sample=True))
print(max(gens, key=score))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[[[Title]]] p-value
[[[Content]]] The p-value is used in the context of null hypothesis testing in order to quantify the statistical significance of a result, the result being the observed value of the chosen statistic T {\displaystyle T}.[note 2] The lower the p-value is, the lower the probability of getting that result if the null hypothesis were true. A result is said to be statistically significant if it allows us to reject the null hypothesis. All other things being equal, smaller p-values are taken as stronger evidence against the null hypothesis.

The following is an interaction between a user and an AI assistant that is related to the above text.

[[[User]]]  Two medical procedures were compared by flipping 2 coins, procedure A assumed to be better and so it was labeled head, while procedure B was labeled as tail for a flip. The coins where then flipped 25 times, with the following results:[{'Tails', 12}, {'Heads', 13}]

Which procedure had better results with statistical signi